In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/us_equities_news_dataset.csv")
df_aapl = df[df["ticker"] == "AAPL"]
df_aapl.to_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/df_aapl.csv", index = False)
'''

'\ndf = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/us_equities_news_dataset.csv")\ndf_aapl = df[df["ticker"] == "AAPL"]\ndf_aapl.to_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/df_aapl.csv", index = False)\n'

In [ ]:
df_aapl = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/df_aapl.csv")

In [ ]:
df_aapl.drop(["id", "ticker", "url", "article_id", "provider"], axis=1, inplace = True)

In [ ]:
df_aapl.iloc[0]['content']

'JPMorgan lifts its Apple  AAPL  2 9   target from  296 to  300 ahead of tomorrow s earnings report \nAnalyst Samik Chatterjee expects an upside quarter driven by better than expected Apple Watch and AirPods sales \nThe analyst expects  88 7B in revenue with  49 7B coming from the iPhone   12 9B from Services  and  11 6B from Wearables \nChatterjee continues to recommend Apple for long term holders but is wary of the  high bar of investor expectations  heading into earnings \nApple has a Bullish average Sell Side rating '

In [ ]:
df_aapl['content'] = df_aapl['content'].apply(lambda x: x.replace(" \n", ". "))

In [ ]:
df_aapl = df_aapl.sort_values('release_date', ignore_index=True)
df_aapl.head()

,title,category,content,release_date
0,Waiting For Direction On The Markets,opinion,This stock market has been a difficult one for...,2012-07-16
1,Mid Year Update U S And Canadian Stock Marke...,opinion,From 2004 to 2010 the TSX 300 Index of leading...,2012-07-19
2,Trade Apple After Earnings,opinion,It may look like a spider web but the mishmas...,2012-07-23
3,Summer Heat Scorches Europe And U S,opinion,Europe flares as summer heat continues Summer...,2012-07-23
4,Apple Earnings Preview Quarterly Dip On Deck,opinion,Last quarter Apple AAPL reported the 2nd bes...,2012-07-23


In [ ]:
df_aapl['category'].unique()

array(['opinion', 'news'], dtype=object)

In [ ]:
df_aapl = df_aapl.join(pd.get_dummies(df_aapl.category))
df_aapl.drop(["category"], axis=1, inplace = True)
df_aapl.head()

,title,content,release_date,news,opinion
0,Waiting For Direction On The Markets,This stock market has been a difficult one for...,2012-07-16,0,1
1,Mid Year Update U S And Canadian Stock Marke...,From 2004 to 2010 the TSX 300 Index of leading...,2012-07-19,0,1
2,Trade Apple After Earnings,It may look like a spider web but the mishmas...,2012-07-23,0,1
3,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,2012-07-23,0,1
4,Apple Earnings Preview Quarterly Dip On Deck,Last quarter Apple AAPL reported the 2nd bes...,2012-07-23,0,1


In [ ]:
%%capture
!pip install sentence-transformers

In [ ]:
%%capture
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentences = df_aapl["title"]
len(sentences)

20231

In [ ]:
title_embeddings = sbert_model.encode(sentences)
#7 secs for 200 sentences
#16+ mins for 20k

In [ ]:
df_aapl["title_embeddings"] = title_embeddings.tolist()

In [ ]:
split_df = pd.DataFrame(df_aapl['title_embeddings'].tolist())

UMAP uses a predefined distance function as a similarity measure. It tries to preserve the distances between points in lower-dimensional space. This makes it perfect for SentenceBERT embedding as the model has a CosineLoss baked in.

In [ ]:
%%capture
!pip install umap-learn 

     |████████████████████████████████| 88 kB 3.9 MB/s 
     |████████████████████████████████| 1.1 MB 39.5 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=04d8dd68a7eea52df070098d00337691219e133706782bc1dfb89545ca2ecf6a
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=6573d263690fabaa83d64ae7bc9a9801c8052ef05000899e73267ad1e2fdf4fc
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


In [ ]:
import umap
reducer = umap.UMAP()

umap_embedding = reducer.fit_transform(split_df)
umap_embedding.shape

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


(20231, 2)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=32)
pca.fit(split_df)
split_df_32 = pca.transform(split_df)

In [ ]:
split_df_32 = pd.DataFrame(split_df_32)
split_df_32

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,-0.046826,-1.141754,7.856352,-1.728148,1.555406,3.766478,0.597022,2.785511,-0.690802,4.756155,...,-0.383450,0.581312,-1.383534,-0.202733,-1.265469,-1.117890,0.733506,1.223844,1.230695,1.081238
1,-1.812753,-0.336095,-3.330870,-4.325975,-0.690286,2.572833,-0.939476,0.362135,1.216966,0.252821,...,1.640829,0.668213,1.217971,1.474376,-0.499962,-1.067962,0.515083,-1.722879,0.302437,1.219896
2,2.394987,2.709535,-0.571670,2.719582,2.794561,7.596699,-0.831986,-0.769614,1.323254,1.332858,...,0.470887,-0.683045,-0.390708,-0.973485,0.220965,0.149232,-0.095951,0.344678,0.451608,-0.067240
3,-3.538170,-2.864984,2.228637,-0.755990,1.179559,-1.576074,1.963962,2.185615,2.872204,1.157740,...,-1.712676,-0.625680,2.426436,-0.441789,0.033768,0.055088,1.017152,2.908112,-0.630118,1.864819
4,-0.459115,2.616327,-6.711671,-1.265890,4.042926,3.572455,-0.061322,-0.003121,1.539257,2.466968,...,0.598755,-0.715471,0.697374,-1.090303,-0.225632,-0.105611,0.211142,0.634802,-0.545757,-0.181289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20226,-0.001814,2.752449,2.966348,-5.003664,-1.864283,-1.402697,-1.467213,-1.434690,1.166473,-2.342139,...,-0.093907,1.743086,1.545030,-1.137670,-0.338241,-1.623253,1.373143,0.103863,-0.570446,-1.750241
20227,0.435077,-0.586416,-3.731032,6.135248,-0.147327,5.731724,1.338139,-1.562765,0.305800,2.099126,...,-1.111335,0.893970,-0.283005,-0.441425,0.263760,0.444213,0.218915,0.279069,0.612630,0.288400
20228,-0.037846,2.110425,-7.242262,1.884527,2.426480,0.149943,-0.685496,-3.520754,-1.259408,-0.231853,...,0.271363,0.001180,1.246393,-1.481226,-0.303964,-1.879630,1.303144,-0.731479,-0.832935,0.033813
20229,-3.676648,3.021006,-6.991304,5.367650,1.076788,0.930420,0.270219,1.927777,-0.420091,0.229712,...,0.762705,0.258323,-0.799669,-0.474644,-0.150216,0.092141,-0.369936,0.363013,1.643978,0.170219


In [ ]:
df_aapl = df_aapl.join(split_df_32)
df_aapl.drop("title", axis=1, inplace=True)

ValueError: ignored

In [ ]:
df_aapl

,title,content,release_date,news,opinion,0,1,2,3,4,...,22,23,24,25,26,27,28,29,30,31
0,Waiting For Direction On The Markets,This stock market has been a difficult one for...,2012-07-16,0,1,-0.046826,-1.141754,7.856352,-1.728148,1.555406,...,-0.383450,0.581312,-1.383534,-0.202733,-1.265469,-1.117890,0.733506,1.223844,1.230695,1.081238
1,Mid Year Update U S And Canadian Stock Marke...,From 2004 to 2010 the TSX 300 Index of leading...,2012-07-19,0,1,-1.812753,-0.336095,-3.330870,-4.325975,-0.690286,...,1.640829,0.668213,1.217971,1.474376,-0.499962,-1.067962,0.515083,-1.722879,0.302437,1.219896
2,Trade Apple After Earnings,It may look like a spider web but the mishmas...,2012-07-23,0,1,2.394987,2.709535,-0.571670,2.719582,2.794561,...,0.470887,-0.683045,-0.390708,-0.973485,0.220965,0.149232,-0.095951,0.344678,0.451608,-0.067240
3,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,2012-07-23,0,1,-3.538170,-2.864984,2.228637,-0.755990,1.179559,...,-1.712676,-0.625680,2.426436,-0.441789,0.033768,0.055088,1.017152,2.908112,-0.630118,1.864819
4,Apple Earnings Preview Quarterly Dip On Deck,Last quarter Apple AAPL reported the 2nd bes...,2012-07-23,0,1,-0.459115,2.616327,-6.711671,-1.265890,4.042926,...,0.598755,-0.715471,0.697374,-1.090303,-0.225632,-0.105611,0.211142,0.634802,-0.545757,-0.181289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20226,The Zacks Analyst Blog Highlights Merck Broa...,For Immediate ReleaseChicago IL January 28 ...,2020-01-28,0,1,-0.001814,2.752449,2.966348,-5.003664,-1.864283,...,-0.093907,1.743086,1.545030,-1.137670,-0.338241,-1.623253,1.373143,0.103863,-0.570446,-1.750241
20227,Stocks Dow Rebounds as Apple Rallies Into Ea...,By Yasin Ebrahim. Investing com The Dow ral...,2020-01-28,1,0,0.435077,-0.586416,-3.731032,6.135248,-0.147327,...,-1.111335,0.893970,-0.283005,-0.441425,0.263760,0.444213,0.218915,0.279069,0.612630,0.288400
20228,Apple earnings 4 99 per share vs 4 55 EPS ...,Apple reported revenue and profit significantl...,2020-01-28,1,0,-0.037846,2.110425,-7.242262,1.884527,2.426480,...,0.271363,0.001180,1.246393,-1.481226,-0.303964,-1.879630,1.303144,-0.731479,-0.832935,0.033813
20229,Apple Earnings Beat in Q1 as iPhone Growth Ret...,By Yasin Ebrahim. Investing com Apple NASDA...,2020-01-28,1,0,-3.676648,3.021006,-6.991304,5.367650,1.076788,...,0.762705,0.258323,-0.799669,-0.474644,-0.150216,0.092141,-0.369936,0.363013,1.643978,0.170219


In [ ]:
content = df_aapl["content"]
len(content)

20231

In [ ]:
content_embeddings = sbert_model.encode(content)
#1 hr 16 minutes

KeyboardInterrupt: ignored

In [ ]:
df_aapl["content_embeddings"] = content_embeddings.tolist()

In [ ]:
split_df = pd.DataFrame(df_aapl['content_embeddings'].tolist())
df_aapl.join(split_df)
df_aapl.drop("content_embeddings", axis=1, inplace=True)

In [ ]:
df_aapl

,title,content,release_date,news,opinion
0,Waiting For Direction On The Markets,This stock market has been a difficult one for...,2012-07-16,0,1
1,Mid Year Update U S And Canadian Stock Marke...,From 2004 to 2010 the TSX 300 Index of leading...,2012-07-19,0,1
2,Trade Apple After Earnings,It may look like a spider web but the mishmas...,2012-07-23,0,1
3,Summer Heat Scorches Europe And U S,Europe flares as summer heat continues Summer...,2012-07-23,0,1
4,Apple Earnings Preview Quarterly Dip On Deck,Last quarter Apple AAPL reported the 2nd bes...,2012-07-23,0,1
...,...,...,...,...,...
20226,The Zacks Analyst Blog Highlights Merck Broa...,For Immediate ReleaseChicago IL January 28 ...,2020-01-28,0,1
20227,Stocks Dow Rebounds as Apple Rallies Into Ea...,By Yasin Ebrahim. Investing com The Dow ral...,2020-01-28,1,0
20228,Apple earnings 4 99 per share vs 4 55 EPS ...,Apple reported revenue and profit significantl...,2020-01-28,1,0
20229,Apple Earnings Beat in Q1 as iPhone Growth Ret...,By Yasin Ebrahim. Investing com Apple NASDA...,2020-01-28,1,0


In [ ]:
df_aapl.to_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/news_embeddings_aapl.csv", index=False)

In [ ]:
df_aapl = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Stonks/data/news_embeddings_aapl.csv")

# Approach 1: Taking average of all embeddings

#Approach 2: Concatenating all sentences and then taking document embeddings